In [ ]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


In [ ]:
# Importing video data from ibm COS


In [1]:
import json
import os
import pickle
import random
import sys
import time
import argparse

import cv2
import ibm_boto3
import numpy as np
import pandas as pd
from ibm_botocore.client import Config
from tqdm import tqdm


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--path', help='path to save vids at')
args = parser.parse_args()


usage: ipykernel_launcher.py [-h] [--path PATH]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-f5c3c334-e013-4c2a-bba1-21653d1554ac.json


SystemExit: 2

/root/w251fp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
# cred_path = "/root/creds/w251-credentials.json"
cred_path = "../creds/w251-credentials.json"
with open(cred_path, "r") as f:
    creds = json.load(f)

In [4]:
auth_endpoint = 'https://iam.bluemix.net/oidc/token'
service_endpoint = 'https://s3.us-east.cloud-object-storage.appdomain.cloud'

# Store relevant details for interacting with IBM COS store and uploading data
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=creds['apikey'],
                         ibm_service_instance_id=creds['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [ ]:
# FIXME: need to figure out how to install aspera for massive download speed
# boost. Surprisingly, the docs suck
#
# from ibm_s3transfer.aspera.manager import AsperaTransferManager
# cos = ibm_boto3.client('s3',
#                          ibm_api_key_id=creds['apikey'],
#                          ibm_service_instance_id=creds['resource_instance_id'],
#                          ibm_auth_endpoint=auth_endpoint,
#                          config=Config(signature_version='oauth'),
#                          endpoint_url=service_endpoint)
# transfer_manager = AsperaTransferManager(cos)

In [5]:
bucket = cos.Bucket('w251-fp-bucket')

In [6]:
files = list(bucket.objects.all())

In [7]:
files

[s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/'),
 s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/SOB_B_A-14-22549AB-100-001.png'),
 s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/SOB_B_A-14-22549AB-100-002.png'),
 s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/SOB_B_A-14-22549AB-100-003.png'),
 s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/SOB_B_A-14-22549AB-100-004.png'),
 s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/SOB_B_A-14-22549AB-100-005.png'),
 s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/SOB_B_A-14-22549AB-100-006.png'),
 s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/SOB_B_A-14-22549AB-100-007.png'),
 s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/SOB_B_A-14-22549AB-100-008.png'),
 s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/SOB_B_A-14-22549AB-100-009.png'),
 s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/SOB_B_A-14-22549AB-100-010.png

In [10]:
f = files[1]
f

s3.ObjectSummary(bucket_name='w251-fp-bucket', key='benign/SOB_B_A-14-22549AB-100-001.png')

In [ ]:
ext = file.key.split('.')[-1]

In [20]:
# savepath = args.path
savepath = '../cancer_data_v1'
bpath = os.path.join(savepath, 'benign')
mpath = os.path.join(savepath, 'malignant')
os.makedirs(bpath, exist_ok=True)
os.makedirs(mpath, exist_ok=True)
filelist = os.listdir(bpath)
filelist.extend(os.listdir(mpath))
filelist

[]

In [22]:
for file in tqdm(files):
    ext = file.key.split('.')[-1]
    if 'png' not in ext:
        continue
    filepath = os.path.join(savepath, file.key.split('/')[-2])
#     os.makedirs(filepath, exist_ok=True)
#     filename = os.path.join(filepath, file.key.split('/')[-1])
    filename = file.key.split('/')[-1]
    if filename in filelist:
        continue
    else:
        filename = os.path.join(filepath, filename)
        bucket.download_file(file.key, filename)

100%|██████████| 7911/7911 [15:39<00:00,  8.42it/s]
